<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/asyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

asyncio를 사용하면 단일 스레드 작업을 병렬로 처리할 수 있다.

In [1]:
import time


def sleep():
    time.sleep(1)


def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end-start}')
    return total


def main():
    start = time.time()

    result1 = sum("A", [1, 2])
    result2 = sum("B", [1, 2, 3])

    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')


if __name__ == "__main__":
    main()


작업중=A, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.000641107559204
작업중=B, number=1, total=1
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.0015058517456055
총합=9, 총시간=5.002198934555054


A 작업 2초, B 작업 3초 등 모두 5초의 시간이 걸리고 sum() 함수를 2번 호출한 결과의 총합은 9임을 알 수 있다.

In [2]:
import asyncio
import time


async def sleep():
    await asyncio.sleep(1)


async def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        await sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end-start}')
    return total


async def main():
    start = time.time()

    task1 = asyncio.create_task(sum("A", [1, 2]))
    task2 = asyncio.create_task(sum("B", [1, 2, 3]))

    await task1
    await task2

    result1 = task1.result()
    result2 = task2.result()

    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     asyncio.run(main())

In [3]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

작업중=A, number=1, total=1
작업중=B, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.0036253929138184
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.0050160884857178
총합=9, 총시간=3.005190849304199


함수를 비동기로 호출하려면 이렇게 def 앞에 async라는 키워드를 넣으면 된다. 그러면 이제 이 함수는 비동기 함수가 된다. 이때 async를 적용한 비동기 함수를 코루틴이라 부른다.

또한, 코루틴 안에서 다른 코루틴을 호출할 때는 await sleep()과 같이 await를 함수명 앞에 붙여 호출해야 한다. 코루틴 수행 중 await 코루틴을 만나면 await로 호출한 코루틴이 종료될 때까지 기다리지 않고 제어권을 메인 스레드나 다른 코루틴으로 넘긴다. 이러한 방식을 넌블록킹(non-blocking)이라 한다. 그리고 호출한 코루틴이 종료되면 이벤트에 의해 다시 그 이후 작업이 수행된다.

A 작업과 B 작업을 교대로 호출한다. (제어권이 await에 의해 계속 바뀐다는 것을 알 수 있다.) 그리고 시간도 5초 걸리던 것이 3초만 걸리게 되므로 A, B 작업이 완전히 비동기적으로 동작했다는 것을 알 수 있다.